In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Define path to TensorBoard log files
logPath = "./tb_logs/"

In [3]:
#   Adds summaries statistics for use in TensorBoard visualization.  
#      From https://www.tensorflow.org/get_started/summaries_and_tensorboard
def variable_summaries(var):
   with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
# Using Interactive session makes it the default sessions so we do not need to pass sess 
sess = tf.InteractiveSession()

In [6]:
# Define placeholders for MNIST input data
with tf.name_scope("MNIST_Input"):
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    y_ = tf.placeholder(tf.float32, [None, 10], name="y_") 

In [7]:
# change the MNIST input data from a list of values to a 28 pixel X 28 pixel X 1 grayscale value cube
#    which the Convolution NN can use.
with tf.name_scope("Input_Reshape"):
    x_image = tf.reshape(x, [-1,28,28,1], name="x_image")
    tf.summary.image('input_img', x_image, 5)

In [8]:
# We are using RELU as our activation function.  These must be initialized to a small positive number 
# and with some noise so you don't end up going to zero when comparing diffs
def weight_variable(shape, name=None):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

In [9]:
def bias_variable(shape, name=None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)


In [10]:
# Convelution and Pooling - we do Convelution, and then pooling to control overfitting
def conv2d(x, W, name=None):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME',name=name)
def max_pool_2x2(x, name=None):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name=name)

In [11]:
# Define the Model

# 1st Convolution layer
with tf.name_scope('Conv1'):
    # 32 features for each 5X5 patch of the image
    with tf.name_scope('weights'):
        W_conv1 = weight_variable([5, 5, 1, 32], name="weight")
        variable_summaries(W_conv1)
    with tf.name_scope('biases'):
        b_conv1 = bias_variable([32], name="bias")
        variable_summaries(b_conv1)
    # Do convolution on images, add bias and push through RELU activation
    conv1_wx_b = conv2d(x_image, W_conv1,name="conv2d") + b_conv1
    tf.summary.histogram('conv1_wx_b', conv1_wx_b)
    h_conv1 = tf.nn.relu(conv1_wx_b, name="relu")
    tf.summary.histogram('h_conv1', h_conv1)
    # take results and run through max_pool
    h_pool1 = max_pool_2x2(h_conv1, name="pool")

In [12]:
# 2nd Convolution layer
with tf.name_scope('Conv2'):
# Process the 32 features from Convolution layer 1, in 5 X 5 patch.  return 64 features weights and biases
    with tf.name_scope('weights'):
        W_conv2 = weight_variable([5, 5, 32, 64], name="weight")
        variable_summaries(W_conv2)
    with tf.name_scope('biases'):
        b_conv2 = bias_variable([64], name="bias")
        variable_summaries(b_conv2)
    # Do convolution of the output of the 1st convolution layer.  Pool results 
    conv2_wx_b = conv2d(h_pool1, W_conv2, name="conv2d") + b_conv2
    tf.summary.histogram('conv2_wx_b', conv2_wx_b)
    h_conv2 = tf.nn.relu(conv2_wx_b, name="relu")
    tf.summary.histogram('h_conv2', h_conv2)
    h_pool2 = max_pool_2x2(h_conv2, name="pool")

In [13]:
with tf.name_scope('FC'):
    # Fully Connected Layer
    W_fc1 = weight_variable([7 * 7 * 64, 1024], name="weight")
    b_fc1 = bias_variable([1024], name="bias")
    #   Connect output of pooling layer 2 as input to full connected layer
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name="relu")

In [14]:
# dropout some neurons to reduce overfitting
keep_prob = tf.placeholder(tf.float32, name="keep_prob")  # get dropout probability as a training input.
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [15]:
with tf.name_scope("Readout"):
# Readout layer
    W_fc2 = weight_variable([1024, 10], name="weight")
    b_fc2 = bias_variable([10], name="bias")

# Define model
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [16]:
# Loss measurement
with tf.name_scope("cross_entropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [17]:
# loss optimization
with tf.name_scope("loss_optimizer"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [18]:
with tf.name_scope("accuracy"):
    # What is correct
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    # How accurate is it?
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
tf.summary.scalar("cross_entropy_scl", cross_entropy)
tf.summary.scalar("training_accuracy", accuracy)

<tf.Tensor 'training_accuracy:0' shape=() dtype=string>

In [20]:
# TB - Merge summaries 
summarize_all = tf.summary.merge_all()

# Initialize all of the variables
sess.run(tf.global_variables_initializer())

# TB - Write the default graph out so we can view it's structure
tbWriter = tf.summary.FileWriter(logPath, sess.graph)

In [21]:
# Train the model
import time

#  define number of steps and how often we display progress
num_steps = 2000
display_every = 100


In [22]:
# Start timer
start_time = time.time()
end_time = time.time()
for i in range(num_steps):
    batch = mnist.train.next_batch(50)
    _, summary = sess.run([train_step, summarize_all], feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})


    # Periodic status display
    if i%display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        end_time = time.time()
        print("step {0}, elapsed time {1:.2f} seconds, training accuracy {2:.3f}%".format(i, end_time-start_time, train_accuracy*100.0))
        # write summary to log
        tbWriter.add_summary(summary,i)


# Display summary 
#     Time to train
end_time = time.time()
print("Total training time for {0} batches: {1:.2f} seconds".format(i+1, end_time-start_time))
#     Accuracy on test data
print("Test accuracy {0:.3f}%".format(accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})*100.0))

step 0, elapsed time 0.52 seconds, training accuracy 22.000%
step 100, elapsed time 30.97 seconds, training accuracy 90.000%
step 200, elapsed time 61.45 seconds, training accuracy 82.000%
step 300, elapsed time 92.28 seconds, training accuracy 86.000%
step 400, elapsed time 123.08 seconds, training accuracy 96.000%
step 500, elapsed time 153.75 seconds, training accuracy 96.000%
step 600, elapsed time 185.11 seconds, training accuracy 96.000%
step 700, elapsed time 214.68 seconds, training accuracy 98.000%
step 800, elapsed time 245.39 seconds, training accuracy 100.000%
step 900, elapsed time 275.19 seconds, training accuracy 96.000%
step 1000, elapsed time 305.10 seconds, training accuracy 100.000%
step 1100, elapsed time 335.95 seconds, training accuracy 100.000%
step 1200, elapsed time 367.38 seconds, training accuracy 100.000%
step 1300, elapsed time 397.87 seconds, training accuracy 98.000%
step 1400, elapsed time 428.29 seconds, training accuracy 98.000%
step 1500, elapsed time